In [1]:
%%capture
! pip install -q --upgrade pypdf llama-index llama-index-llms-gemini llama-index-embeddings-gemini

In [2]:
## get the data
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-06-30 06:52:54--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  39.1MB/s    in 0.4s    

2024-06-30 06:52:55 (39.1 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [3]:
import os
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
from llama_index.core import Settings
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import StorageContext
from llama_index.core import SimpleDirectoryReader
from llama_index.core.query_engine import CitationQueryEngine
import nest_asyncio

nest_asyncio.apply()

In [5]:
##Load the data
documents = SimpleDirectoryReader(input_files=['metagpt.pdf']).load_data()

In [6]:
gemini_embedding_model = GeminiEmbedding(model_name="models/embedding-001")
llm = Gemini(model_name="models/gemini-1.5-flash-latest")

In [7]:
# Set Global settings
Settings.llm = llm
Settings.embed_model = gemini_embedding_model

In [8]:
index = VectorStoreIndex(documents)

In [9]:
##Create the CitationQueryEngine with Default Arguments
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    #default is 512
    citation_chunk_size=512,
)

response = query_engine.query("How do agents share information with other agents?")
print(response)

Agents share information with other agents by complying with development and communication protocols defined in MetaGPT. [2] 



In [10]:
print(len(response.source_nodes))

5


In [11]:
## Inspecting the source
print(response.source_nodes[0].node.get_text())

Source 1:
Preprint
Figure 12: The program call flow for “recommendation engine development” generated by the
architect agent ( zoom in for a better view ).
ware design scenarios and standard operating procedures (SOPs) where effective communication is
essential.
27



In [12]:
print(response.source_nodes[1].node.get_text())

Source 2:
Preprint
and pay according to their usage. Moreover, users can purchase additional capabilities to expand the
plug-and-play functions of their existing agents. This allows users to gradually upgrade their agents.
Within the MetaGPT framework, AgentStore can support the collaboration of various agents. Users
can collect several agents together to carry out more complex tasks or projects, and all the agents
share and comply with development and communication protocols defined in MetaGPT.
Figure 6: AgentStore is a platform dedicated to serving users in the creation and development of
agents within the MetaGPT framework. This platform provides users with an operational interface,
allowing users to easily manage a variety of agents with different emotions, personalities, and capa-
bilities for specific tasks.
16



In [13]:
#Adjusting the Setting
query_engine = CitationQueryEngine.from_args(
    index,
    # increase the citation chunk size!
    citation_chunk_size=1024,
    similarity_top_k=3,
)

In [14]:
response = query_engine.query("How do agents share information with other agents?")

In [15]:
print(response)

Agents share and comply with development and communication protocols defined in MetaGPT. [2] 



In [16]:
# should be less source nodes now!
print(len(response.source_nodes))

4


In [17]:
##Inspecting the actual source
print(response.source_nodes[0].node.get_text())

Source 1:
Preprint
Figure 12: The program call flow for “recommendation engine development” generated by the
architect agent ( zoom in for a better view ).
ware design scenarios and standard operating procedures (SOPs) where effective communication is
essential.
27

